### Reading TGeomPoint data
with movingpandas

In [22]:
import datetime

from sqlalchemy import func
from sqlalchemy import Column, Integer, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from mobilitydb_sqlalchemy import TGeomPoint

from shapely.geometry import Point

In [5]:
engine = create_engine("postgresql://docker:docker@db:5432/mobilitydb", echo=True)
session = sessionmaker(bind=engine)()

In [8]:
Base = declarative_base()

class Trips(Base):
    __tablename__ = "trips_test_002"
    car_id = Column(Integer, primary_key=True)
    trip_id = Column(Integer, primary_key=True)
    trip = Column(TGeomPoint(use_movingpandas=True))
    
Base.metadata.create_all(engine)

2021-04-23 15:47:14,764 INFO sqlalchemy.engine.Engine select version()
2021-04-23 15:47:14,765 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-23 15:47:14,768 INFO sqlalchemy.engine.Engine select current_schema()
2021-04-23 15:47:14,769 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-23 15:47:14,770 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-04-23 15:47:14,771 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-23 15:47:14,774 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-23 15:47:14,775 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-04-23 15:47:14,777 INFO sqlalchemy.engine.Engine [generated in 0.00171s] {'name': 'trips_test_002'}
2021-04-23 15:47:14,780 INFO sqlalchemy.engine.Engine COMMIT


#### Value at a given timestamp

SQL query:
``` 
SELECT CarId,
       ST_AsText(valueAtTimestamp(Trip, timestamptz '2012-01-01 08:10:00'))
FROM Trips;
```

In [17]:
Trips.metadata

MetaData()

In [19]:
session.query(
    Trips.car_id,
    func.asText(
        func.valueAtTimestamp(Trips.trip, datetime.datetime(2012, 1, 1, 8, 10, 0))
    ),
).all()

2021-04-23 15:54:17,972 INFO sqlalchemy.engine.Engine SELECT trips_test_002.car_id AS trips_test_002_car_id, asText(valueAtTimestamp(trips_test_002.trip, %(valueAtTimestamp_1)s)) AS "asText_1" 
FROM trips_test_002
2021-04-23 15:54:17,974 INFO sqlalchemy.engine.Engine [cached since 373.1s ago] {'valueAtTimestamp_1': datetime.datetime(2012, 1, 1, 8, 10)}


[(1, 'POINT(2 0)')]

#### Restriction to a given value

SQL query:
``` 
SELECT CarId, 
       asText(atValue(Trip, 'Point(2 0)'))
FROM Trips;
```

In [23]:
session.query(
    Trips.car_id,
    func.asText(func.atValue(Trips.trip, Point(2, 0).wkt)),
).all()

2021-04-23 15:55:46,836 INFO sqlalchemy.engine.Engine SELECT trips_test_002.car_id AS trips_test_002_car_id, asText(atValue(trips_test_002.trip, %(atValue_1)s)) AS "asText_1" 
FROM trips_test_002
2021-04-23 15:55:46,837 INFO sqlalchemy.engine.Engine [generated in 0.00163s] {'atValue_1': 'POINT (2 0)'}


[(1, '{[POINT(2 0)@2012-01-01 08:10:00+00]}')]

#### Restriction to a period

SQL query:
``` 
SELECT CarId, 
       asText(atPeriod(Trip, '[2012-01-01 08:05:00,2012-01-01 08:10:00]'))
FROM Trips;
```

In [24]:
session.query(
    Trips.car_id,
    func.asText(
        func.atPeriod(Trips.trip, "[2012-01-01 08:05:00,2012-01-01 08:10:00]")
    ),
).all()

2021-04-23 15:56:18,461 INFO sqlalchemy.engine.Engine SELECT trips_test_002.car_id AS trips_test_002_car_id, asText(atPeriod(trips_test_002.trip, %(atPeriod_1)s)) AS "asText_1" 
FROM trips_test_002
2021-04-23 15:56:18,462 INFO sqlalchemy.engine.Engine [generated in 0.00174s] {'atPeriod_1': '[2012-01-01 08:05:00,2012-01-01 08:10:00]'}


[(1, '[POINT(1 0)@2012-01-01 08:05:00+00, POINT(2 0)@2012-01-01 08:10:00+00]')]

#### Temporal distance

SQL query:
``` 
SELECT T1.CarId, 
       T2.CarId, 
       T1.Trip <-> T2.Trip
FROM Trips T1, 
     Trips T2
WHERE T1.CarId < T2.CarId;
```

In [25]:
session.query(
    T1.c.car_id,
    T2.c.car_id,
    T1.c.trip.distance(T2.c.trip),
) \
.filter(T1.c.car_id < T2.c.car_id,)
.all()

SyntaxError: invalid syntax (<ipython-input-25-74ec803d62fb>, line 7)